In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ta.volatility import AverageTrueRange
import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from scipy.stats import normaltest
from statsmodels.tsa.stattools import acf,pacf
#from pmdarima.arima import auto_arima
import scipy.interpolate as sci
import scipy.optimize as sco
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import scipy.optimize as sco
from tqdm import tqdm

In [2]:
TP = 400
SL = 200

In [3]:
df_daily = pd.read_csv("../src/daily_data/Si_230101_240831.csv", sep=";")
df_min = pd.read_excel("../src/stock_data.xlsx", sheet_name="Si")

In [4]:
def prepare_df(df: pd.DataFrame):
    df = df.rename(columns={col: col.replace("<", "").replace(">", "") for col in df.columns})
    df.drop(["TICKER", "PER"], axis=1, inplace=True)
    df["DATE"] = pd.to_datetime(df["DATE"], format="%y%m%d")
    return df

In [5]:
def get_atr(data, window=14):
    atr = AverageTrueRange(
        high=data["HIGH"],
        low=data["LOW"],
        close=data["CLOSE"],
        window=window
    ).average_true_range()
    return atr

In [6]:
df_daily = prepare_df(df_daily)
df_min = prepare_df(df_min)

In [7]:
myatr = get_atr(df_daily)
df_daily["ATR"] = round(myatr, 2)

In [8]:
df_daily.head(3)

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,ATR
0,2023-01-03,0,70105,72280,69873,72236,920700,0.0
1,2023-01-04,0,72227,72600,71310,72530,846368,0.0
2,2023-01-05,0,72510,72844,71675,72563,708928,0.0


In [9]:
df_daily = df_daily.loc[df_daily["ATR"] > 0]

In [10]:
df_daily["SL_SIZE"] = round(df_daily["ATR"] / 3, 0)
df_daily["TP_SIZE"] = round(df_daily["ATR"] * 2 / 3, 0)
# df_daily["SL_SIZE"] = 200
# df_daily["TP_SIZE"] = 400
df_daily["LONG_SL"] = df_daily["CLOSE"] - df_daily["SL_SIZE"]
df_daily["LONG_TP"] = df_daily["CLOSE"] + df_daily["TP_SIZE"]
df_daily["SHORT_SL"] = df_daily["CLOSE"] + df_daily["SL_SIZE"]
df_daily["SHORT_TP"] = df_daily["CLOSE"] - df_daily["TP_SIZE"]

for col in ["LONG_SL", "LONG_TP", "SHORT_SL", "SHORT_TP"]:
    df_daily[col] = df_daily[col].shift()

In [11]:
df_daily = df_daily.dropna()

In [12]:
df_min = df_min.merge(df_daily[[
    "DATE",
    "ATR",
    "SL_SIZE",
    "TP_SIZE",
    "LONG_SL",
    "LONG_TP",
    "SHORT_SL",
    "SHORT_TP"
]], how="left", on="DATE")

In [13]:
df_min = df_min.dropna()

In [14]:
df_daily["PROFIT_LONG"] = 0
df_daily["PROFIT_SHORT"] = 0

In [15]:
df_daily = df_daily.loc[df_daily["DATE"].isin(df_min["DATE"].unique())]
df_min = df_min.loc[df_min["DATE"].isin(df_daily["DATE"].unique())]

In [16]:
df_min.head()

,DATETIME_KEY,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,ATR,SL_SIZE,TP_SIZE,LONG_SL,LONG_TP,SHORT_SL,SHORT_TP
2436,230123_90000,2023-01-23,90000,69498,69500,69365,69400,7161,1266.82,422.0,845.0,69033.0,70318.0,69889.0,68604.0
2437,230123_90500,2023-01-23,90500,69400,69430,69388,69388,3497,1266.82,422.0,845.0,69033.0,70318.0,69889.0,68604.0
2438,230123_91000,2023-01-23,91000,69393,69400,69340,69340,3872,1266.82,422.0,845.0,69033.0,70318.0,69889.0,68604.0
2439,230123_91500,2023-01-23,91500,69340,69357,69204,69224,9355,1266.82,422.0,845.0,69033.0,70318.0,69889.0,68604.0
2440,230123_92000,2023-01-23,92000,69224,69246,69209,69210,6158,1266.82,422.0,845.0,69033.0,70318.0,69889.0,68604.0


In [17]:
dates = df_min["DATE"].unique()

for d in tqdm(dates):
    # минимальное время в дату
    min_time = df_min.loc[df_min["DATE"] == d, "TIME"].min()
    # считаем построчно срабатывание TP ил SL лонга
    df_day = df_min.loc[df_min["DATE"] == d].copy()
    for i in range(0, len(df_day)):
        if np.array(df_day["CLOSE"])[i] >= np.array(df_day["LONG_TP"])[i]:
            df_daily.loc[df_daily["DATE"] == d, "PROFIT_LONG"] = np.array(df_day["TP_SIZE"])[0]
        elif np.array(df_day["CLOSE"])[i] <= np.array(df_day["LONG_SL"])[i]:
            df_daily.loc[df_daily["DATE"] == d, "SL_SIZE"] = np.array(df_day["SL_SIZE"])[0]
        # то же самое по шорту
        elif np.array(df_day["CLOSE"])[i] <= np.array(df_day["SHORT_TP"])[i]:
            df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = np.array(df_day["TP_SIZE"])[0]
        elif np.array(df_day["CLOSE"])[i] >= np.array(df_day["SHORT_SL"])[i]:
            df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = np.array(df_day["SL_SIZE"])[0]
        # если не сработали триггеры
        else:
            profit = np.array(df_day.loc[df_day["TIME"] == 230000, "CLOSE"])[0] - np.array(df_day["CLOSE"])[0]
            if profit > 0:
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_LONG"] = profit
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = profit * (-1)
            else:
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = profit
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_LONG"] = profit * (-1)

100%|██████████| 409/409 [01:05<00:00,  6.25it/s]


In [18]:
df_daily

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,ATR,SL_SIZE,TP_SIZE,LONG_SL,LONG_TP,SHORT_SL,SHORT_TP,PROFIT_LONG,PROFIT_SHORT
14,2023-01-23,0,69498,69945,68921,69799,967899,1266.82,422.0,845.0,69033.0,70318.0,69889.0,68604.0,425,-425
15,2023-01-24,0,69790,69791,69364,69570,658238,1207.41,402.0,805.0,69377.0,70644.0,70221.0,68954.0,119,-119
16,2023-01-25,0,69570,69987,69421,69857,767748,1161.59,387.0,774.0,69168.0,70375.0,69972.0,68765.0,204,-204
17,2023-01-26,0,69857,70266,69276,69670,1189573,1149.33,383.0,766.0,69470.0,70631.0,70244.0,69083.0,315,-315
18,2023-01-27,0,69675,70175,69649,70050,821428,1104.81,368.0,737.0,69287.0,70436.0,70053.0,68904.0,328,-328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,2024-08-26,0,88694,89651,88213,89113,659823,1285.31,428.0,857.0,88178.0,89452.0,89028.0,87754.0,716,428
419,2024-08-27,0,89105,89789,88760,89600,526971,1267.00,422.0,845.0,88685.0,89970.0,89541.0,88256.0,214,422
420,2024-08-28,0,89600,89967,88934,89412,522477,1250.29,417.0,834.0,89178.0,90445.0,90022.0,88755.0,143,-143
421,2024-08-29,0,89413,90125,89303,89977,653571,1219.70,407.0,813.0,88995.0,90246.0,89829.0,88578.0,486,407


In [19]:
df_daily["TOTAL_PROFIT"] = df_daily["PROFIT_LONG"] + df_daily["PROFIT_SHORT"]

In [22]:
df_daily["TOTAL_PROFIT"].sum() * 2 - 40 * len(df_daily)
# 147364
# 400-800 - 143745

278368